## A workbook for mapping and concensus generation for multiple segments

### Generate the reference

```bash
efetch -db nuccore -id acc1 acc2 acc3 -format fasta > ref.fasta

```

The command above will fetch multiple sequences in fasta format from NCBI

In [2]:
!cat refs/genome.fasta | grep \>

>X14383.1 Bunyamwera virus L protein RNA, complete cds
>M11852.1 Bunyamwera virus M polyprotein gene, complete cds
>D00353.1 Bunyamwera virus genes for N and NSs protein, complete cds


In [8]:
!ls -lh | grep fastq

-rw-rw-r-- 1 armand armand 7.6M Apr 26 10:53 bc1_to_7.fastq


## Map to ref with with minimap2

In [9]:
%%bash 
minimap2 -ax map-ont refs/genome.fasta bc1_to_7.fastq | samtools sort -o all_combined.bam
samtools index all_combined.bam

[M::mm_idx_gen::0.001*2.16] collected minimizers
[M::mm_idx_gen::0.002*2.43] sorted minimizers
[M::main::0.002*2.41] loaded/built the index for 3 target sequence(s)
[M::mm_mapopt_update::0.002*2.29] mid_occ = 10
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 3
[M::mm_idx_stat::0.002*2.23] distinct minimizers: 2297 (100.00% are singletons); average occurrences: 1.000; average spacing: 5.352; total length: 12294
[M::worker_pipeline::0.195*2.76] mapped 5851 sequences
[M::main] Version: 2.26-r1175
[M::main] CMD: minimap2 -ax map-ont refs/genome.fasta bc1_to_7.fastq
[M::main] Real time: 0.196 sec; CPU: 0.540 sec; Peak RSS: 0.022 GB


## Create a bed file for the primers

In this case the primers are in an excell spreadsheet

In [10]:
import pandas as pd

In [48]:
primers_not_used = ["BUNV_M_msq_R2", "BUNV_L_msq_F1"]
primerDf = pd.read_excel('BUNV primer sets_updated 240328.xlsx', skiprows=2, header=None)
primerDf = primerDf[~primerDf[0].isin(primers_not_used)]
primerDf.dropna(inplace=True, subset=[0])
primerDf.shape

(21, 11)

In [38]:
with open("primers.fasta", "w") as f:
    for r in range(primerDf.shape[0]):
        header = primerDf.iloc[r, 0]
        sequence = primerDf.iloc[r, 2]
        f.write(f'>{header}\n')
        f.write(f'{sequence}\n')
!cat primers.fasta

>BUNV_S_F1
AGTAGTGTACTCCACACTA
>BUNV_S_msq_F2
CTCAGTGGATTCCTTGC
>BUNV_S_msq_R1
GTTCCTARGAACATYTCTGATC
>BUNV_S_msq_R1_alt
GTHCCTARGAACATYTCWGABCC
>BUNV_S_R2
AGTAGTGTGCTCCACC
>BUNV_M_F1
AGTAGTGTACTACCGATACA
>BUNV_M_msq_F2
GCAACGARTGCAATATGTAC
>BUNV_M_msq_F3
CTCCCGCATATAAAACCAG
>BUNV_M_msq_R1
GGTGCAATTAAAGGCCC
>BUNV_M_msq_R2_alt
CTGGCTGTTATGAATGCC
>BUNV_M_R3
AGTAGTGTGCTACCGATA
>BUNV_L_msq_F1_alt
CAGTAGGAGTATGGAGGAC
>BUNV_L_msq_F2
GATTGCCAGGTCAACTG
>BUNV_L_msq_F3
GAAACARCATGTCAATCTACC
>BUNV_L_msq_F4
GGAAGGAAATACCAGTTGAG
>BUNV_L_msq_F5
GAAGAGGCAAGCAGATTG
>BUNV_L_msq_R1
CTATACATTGCCAGAAGCAAG
>BUNV_L_msq_R2
GCCCTATAATTGCAATGTTG
>BUNV_L_msq_R3
CAAGCCGACTAATGCTATC
>BUNV_L_msq_R4
GTCTGCCAGTTATTCCATG
>BUNV_L_R5
AGTAGTGTGCTCCTACATA


In [47]:
%%bash

bwa index refs/genome.fasta
bwa mem -k 4 -T 12 refs/genome.fasta primers.fasta | samtools sort -o primers.bam
samtools index primers.bam
bedtools bamtobed -i primers.bam > primers.bed

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1198-dirty
[main] CMD: bwa index refs/genome.fasta
[main] Real time: 0.362 sec; CPU: 0.005 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 21 sequences (402 bp)...
[M::mem_process_seqs] Processed 21 reads in 0.002 CPU sec, 0.002 real sec
[main] Version: 0.7.17-r1198-dirty
[main] CMD: bwa mem -k 4 -T 12 refs/genome.fasta primers.fasta
[main] Real time: 0.002 sec; CPU: 0.003 sec


You may have to tweak the -k and -T parameters

In [49]:
!wc -l primers.bed

21 primers.bed


We had 21 primers in our fasta file and the same number in our bed file

## Trim the primers with ivar and confirm in something like IGV

In [50]:
!ivar trim -i all_combined.bam -b primers.bed -m 10 -q 0 | samtools sort -o all_combined_ptimmed.bam
!samtools index all_combined_ptimmed.bam

Found 21 primers in BED file
Reading from all_combined.bam

-------
Results: 
Primer Name	Read Count
BUNV_L_msq_F1_alt	110
BUNV_L_msq_F2	394
BUNV_L_msq_R1	127
BUNV_L_msq_F3	130
BUNV_L_msq_R2	158
BUNV_L_msq_F4	13
BUNV_L_msq_R3	46
BUNV_L_msq_F5	155
BUNV_L_msq_R4	155
BUNV_L_R5	175
BUNV_M_F1	219
BUNV_M_msq_F2	18
BUNV_M_msq_R1	48
BUNV_M_msq_F3	145
BUNV_M_msq_R2_alt	19
BUNV_M_R3	309
BUNV_S_F1	0
BUNV_S_msq_F2	1317
BUNV_S_msq_R1_alt	177
BUNV_S_msq_R1	0
BUNV_S_R2	1355

Trimmed primers from 88.72% (3161) of reads.
0% (0) of reads were quality trimmed below the minimum length of 10 bp and were not written to file.
11.28% (402) of reads that started outside of primer regions were not written to file
2871 unmapped reads were not written to file.
100% (3563) of reads had their insert size smaller than their read length


!['igv img primers trimmed'](igv_snapshot_primers_trimmed.png)

Above you can see the effect of trimming primers on depth

## Consensus calling

In [5]:
%%bash

samtools consensus -f fasta all_combined_ptimmed.bam -o all_combined_ptimmed_draft.fasta

## Polish with medaka

In [8]:
!medaka_consensus -i bc1_to_7.fastq -d all_combined_ptimmed_draft.fasta -m r1041_e82_400bps_sup_v4.2.0 -f

TF_CPP_MIN_LOG_LEVEL is set to '3'
Checking program versions
This is medaka 1.11.3
Program    Version    Required   Pass     
bcftools   1.17       1.11       True     
bgzip      1.17       1.11       True     
minimap2   2.26       2.11       True     
samtools   1.18       1.11       True     
tabix      1.17       1.11       True     
[10:28:30 - MdlStrTF] Successfully removed temporary files from /tmp/tmpycxtx3_r.
[10:28:30 - MdlStrTF] Successfully removed temporary files from /tmp/tmp5csuqof2.
Aligning basecalls to draft
Creating fai index file /storage/1T/NGS2/ONT_sequencing/BUNW-WGS/all_combined_ptimmed_draft.fasta.fai
Creating mmi index file /storage/1T/NGS2/ONT_sequencing/BUNW-WGS/all_combined_ptimmed_draft.fasta.map-ont.mmi
[M::mm_idx_gen::0.001*1.81] collected minimizers
[M::mm_idx_gen::0.002*2.11] sorted minimizers
[M::main::0.002*1.71] loaded/built the index for 3 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 3
[M::mm_idx_stat::0.002*1.69] 